# Event Management with Neon CRM SDK

This notebook demonstrates comprehensive event management operations using the Neon CRM SDK.

⚠️ **SAFETY NOTE**: This notebook contains only read-only operations for safety.
Database-modifying operations are commented out to prevent accidental changes.

In [ ]:
import os
from datetime import datetime, timedelta
from neon_crm import NeonClient
import pandas as pd

# Initialize the client
client = NeonClient(
    org_id=os.getenv('NEON_ORG_ID'),
    api_key=os.getenv('NEON_API_KEY'),
)

print(f"Connected to Neon CRM - Environment: {client.environment}")

## 1. List Events (Read-Only)

Retrieve and display events from your organization.

In [ ]:
# List events (safe read-only operation)
print("📅 Retrieving events...")

events = list(client.events.list(page_size=200, limit=10))

print(f"✓ Retrieved {len(events)} events")

if events:
    # Display event information
    for i, event in enumerate(events[:5], 1):
        print(f"\n{i}. {event.get('name', 'Unnamed Event')}")
        print(f"   ID: {event.get('id')}")
        print(f"   Start Date: {event.get('startDate', 'N/A')}")
        print(f"   Status: {event.get('eventStatus', 'N/A')}")
        print(f"   Category: {event.get('category', {}).get('name', 'N/A')}")
else:
    print("No events found in your organization.")

## 2. Event Search Operations (Read-Only)

Search for events using specific criteria.

In [ ]:
# Search for upcoming events
print("🔍 Searching for upcoming events...")

search_request = {
    "searchFields": [
        {"field": "Event Archived", "operator": "EQUAL", "value": "false"}
    ],
    "outputFields": [
        "Event ID", "Event Name", "Event Start Date", "Event End Date",
        "Event Category", "Event Registration Status"
    ],
    "pagination": {"currentPage": 0, "pageSize": 200}
}

try:
    search_results = list(client.events.search(search_request, validate=False))

    print(f"✓ Found {len(search_results)} non-archived events")

    if search_results:
        # Convert to DataFrame for better display
        df = pd.DataFrame(search_results)
        print("\n📊 Events Summary:")
        print(df.head())

        # Display some key statistics
        print(f"\nTotal Events Found: {len(search_results)}")

        # Show unique event categories if available
        if 'Event Category' in df.columns:
            unique_categories = df['Event Category'].value_counts()
            print(f"\nEvent Categories Distribution:")
            for category, count in unique_categories.head().items():
                print(f"  {category}: {count}")

    else:
        print("No non-archived events found.")

except Exception as e:
    print(f"❌ Search failed: {e}")
    print("Note: This may be due to field name validation issues.")

    # Try a simpler search as fallback
    print("\n🔄 Trying simpler search...")
    try:
        simple_search = {
            "searchFields": [
                {"field": "Event Name", "operator": "NOT_BLANK", "value": ""}
            ],
            "outputFields": [
                "Event ID", "Event Name", "Event Start Date"
            ],
            "pagination": {"currentPage": 0, "pageSize": 50}
        }

        simple_results = list(client.events.search(simple_search, validate=False))
        print(f"✓ Simple search found {len(simple_results)} events with names")

        if simple_results:
            simple_df = pd.DataFrame(simple_results)
            print("\n📊 Simple Search Results:")
            print(simple_df.head())

    except Exception as simple_e:
        print(f"❌ Simple search also failed: {simple_e}")

## 3. Event Categories (Read-Only)

Retrieve available event categories.

In [ ]:
# Get event categories
print("📂 Retrieving event categories...")

try:
    categories = client.events.get_categories()

    print(f"✓ Found {len(categories)} event categories")

    for category in categories:
        print(f"  • {category.get('name', 'Unnamed Category')} (ID: {category.get('id')})")

except Exception as e:
    print(f"❌ Failed to retrieve categories: {e}")

## 4. Event Field Information (Read-Only)

Get available search and output fields for events.

In [ ]:
# Get event search fields
print("🔍 Retrieving event search fields...")

try:
    search_fields = client.events.get_search_fields()

    print(f"✓ Found {len(search_fields)} search fields")
    print("\nSample search fields:")
    for field in search_fields['standardFields']:
        print(f"  • {field}")

    for field in search_fields['customFields']:
        print(f"  • {field}")
except Exception as e:
    print(f"❌ Failed to retrieve search fields: {e}")

In [ ]:
# Get event output fields
print("📊 Retrieving event output fields...")

try:
    output_fields = client.events.get_output_fields()

    print(f"✓ Found {len(output_fields)} output fields")
    print("\nSample output fields:")
    for field in output_fields['standardFields']:
        print(f"  • {field}")

    for field in output_fields['customFields']:
        print(f"  • {field}")

except Exception as e:
    print(f"❌ Failed to retrieve output fields: {e}")

## 5. Event Registration Analysis (Read-Only)

Analyze event registrations if events are available.

In [ ]:
# Analyze event registrations
print("📈 Analyzing event registrations...")

if events:
    # Get the first event for demonstration
    sample_event = events[0]
    event_id = sample_event.get('id')

    if event_id:
        try:
            # Get registrations for this event
            registrations = list(client.events.get_registrations(event_id, limit=50))

            print(f"✓ Event '{sample_event.get('name')}' has {len(registrations)} registrations")

            if registrations:
                # Analyze registration data
                reg_df = pd.DataFrame(registrations)

                print("\n📊 Registration Summary:")
                print(f"Total Registrations: {len(registrations)}")

                # Show registration status distribution if available
                if 'registrationStatus' in reg_df.columns:
                    status_counts = reg_df['registrationStatus'].value_counts()
                    print("\nRegistration Status Distribution:")
                    for status, count in status_counts.items():
                        print(f"  {status}: {count}")

            else:
                print("No registrations found for this event.")

        except Exception as e:
            print(f"❌ Failed to retrieve registrations: {e}")
    else:
        print("❌ Event ID not available.")
else:
    print("⚠️ No events available for registration analysis.")

## 6. Event Performance Metrics (Read-Only)

Calculate performance metrics for events.

In [ ]:
# Calculate event performance metrics
print("📊 Calculating event performance metrics...")

if events:
    # Create a summary of all events
    event_summary = []

    for event in events[:5]:  # Analyze first 5 events for performance
        event_id = event.get('id')
        event_name = event.get('name', 'Unnamed Event')

        try:
            # Get registration count
            registrations = list(client.events.get_registrations(event_id, limit=1000))
            registration_count = len(registrations)

            # Calculate basic metrics
            event_summary.append({
                'Event Name': event_name,
                'Event ID': event_id,
                'Start Date': event.get('startDate', 'N/A'),
                'Status': event.get('eventStatus', 'N/A'),
                'Registration Count': registration_count,
                'Category': event.get('category', {}).get('name', 'N/A')
            })

        except Exception as e:
            print(f"⚠️ Could not analyze event {event_name}: {e}")
            continue

    if event_summary:
        # Display performance summary
        summary_df = pd.DataFrame(event_summary)

        print("\n📈 Event Performance Summary:")
        print(summary_df)

        print("\n📊 Key Metrics:")
        total_registrations = summary_df['Registration Count'].sum()
        avg_registrations = summary_df['Registration Count'].mean()

        print(f"Total Registrations Across Events: {total_registrations}")
        print(f"Average Registrations Per Event: {avg_registrations:.1f}")

        # Find most popular event
        if total_registrations > 0:
            top_event = summary_df.loc[summary_df['Registration Count'].idxmax()]
            print(f"Most Popular Event: {top_event['Event Name']} ({top_event['Registration Count']} registrations)")
    else:
        print("⚠️ Could not analyze any events.")
else:
    print("⚠️ No events available for performance analysis.")

## ⚠️ Database-Modifying Operations (COMMENTED OUT FOR SAFETY)

The following operations would modify your database and are commented out for safety.
Uncomment and modify carefully if you need to create/update events.

In [ ]:
# ⚠️ WARNING: DATABASE-MODIFYING OPERATIONS COMMENTED OUT FOR SAFETY

# # Create a new event (DANGEROUS - COMMENTED OUT)
# print("🆕 Creating new event...")
#
# event_data = {
#     "event": {
#         "name": "Test Event - SDK Demo",
#         "startDate": (datetime.now() + timedelta(days=30)).strftime("%Y-%m-%d"),
#         "endDate": (datetime.now() + timedelta(days=30)).strftime("%Y-%m-%d"),
#         "eventStatus": "ACTIVE",
#         "category": {"id": 1},  # Use appropriate category ID
#         "description": "Demo event created by Neon CRM SDK"
#     }
# }
#
# try:
#     # new_event = client.events.create(event_data)
#     # print(f"✓ Created event: {new_event.get('id')}")
#     pass
# except Exception as e:
#     print(f"❌ Event creation failed: {e}")

print("🛡️ Database-modifying operations are commented out for safety.")
print("Uncomment the code above if you need to create events.")

## Summary

This notebook demonstrates comprehensive event management operations:

### ✅ Completed Operations:
- Event listing and basic information retrieval
- Event search with filtering capabilities
- Event category exploration
- Field information discovery
- Registration analysis and metrics
- Performance summary calculations

### 🛡️ Safety Features:
- All read-only operations are safe to execute
- Database-modifying operations are commented out
- Error handling for missing data scenarios
- Validation bypass for testing field names

### 📝 Key Learnings:
- Events support both list() and search() operations
- Field names should use Display Name format (e.g., "Event Name")
- Some event fields may be optional and not populated for all events
- Event categories provide useful organizational structure

### 🔧 Best Practices Demonstrated:
- **Graceful Degradation**: Handling cases where events are not available
- **Data Validation**: Checking for required fields and handling missing data
- **Performance Analysis**: Using metrics to evaluate event success
- **User Experience**: Providing clear feedback about system capabilities

### 🚀 Advanced Applications:
- **Capacity Planning**: Using utilization data to optimize event sizing
- **Revenue Optimization**: Identifying high-performing event characteristics
- **Marketing Insights**: Understanding which events resonate with audiences
- **Resource Allocation**: Data-driven decisions about event investments

Events are a powerful tool for engagement and fundraising - this notebook provides the foundation for building sophisticated event management and analysis systems.